##### https://diane-space.tistory.com/285
##### https://techblog-history-younghunjo1.tistory.com/373?category=922523
##### https://wpaud16.tistory.com/67

In [57]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [18]:
# 데이터 불러오기 
xy = pd.read_csv('./day_stock_data.csv', encoding='utf-8')
xy = xy[::-1] # 역순으로 저장 

In [19]:
xy = xy.drop(['주가변동'], axis=1)

In [20]:
xy = xy.drop(['Unnamed: 0'], axis=1)

In [21]:
xy = xy.drop(['날짜'], axis=1)

In [22]:
xy = xy.drop(labels=range(0,10100), axis=0)

In [56]:
xy.head()

,시가,고가,저가,종가,거래량
11337,68300,68700,67600,67900,11670251
11336,66600,67900,66600,67600,15680447
11335,67100,67400,66100,66300,11937555
11334,65300,66700,65200,66500,14551536
11333,65200,65500,64900,64900,16414188


In [58]:
scaler_ = MinMaxScaler()
scaler_.fit(xy)
xy_scaled = scaler_.transform(xy)

In [59]:
xy_scaled_df = pd.DataFrame(xy_scaled, columns=xy.columns)

In [60]:
xy_scaled_df

,시가,고가,저가,종가,거래량
0,0.583728,0.525338,0.584046,0.568627,0.129230
1,0.551561,0.511824,0.565052,0.563025,0.173636
2,0.561022,0.503378,0.555556,0.538749,0.132190
3,0.526963,0.491554,0.538462,0.542484,0.161136
4,0.525071,0.471284,0.532764,0.512605,0.181762
...,...,...,...,...,...
1233,0.157237,0.144595,0.167331,0.156303,0.104344
1234,0.150804,0.144932,0.158974,0.150327,0.235619
1235,0.164806,0.162500,0.166192,0.152194,0.259240
1236,0.152696,0.159122,0.161254,0.178711,0.216846


In [42]:
# # 정규화 함수 
# def MinMaxScaler(data):
#     denom = np.max(data,0)-np.min(data,0)
#     nume = data-np.min(data,0)
#     return nume/denom

# # 정규화 되돌리기 함수 
# def back_MinMax(data,value):
#     diff = np.max(data,0)-np.min(data,0)
#     back = value * diff + np.min(data,0)
#     return back 

In [77]:
 X_train, y_train = xy_scaled_df.iloc[:, :-1].values, xy_scaled_df.iloc[:, [-1]].values
# X_train = X_train.astype(int)
# y_train - y_train.astype(int)
# X_train, y_train

In [68]:
seqLength = 7 # window size = row 수
dataDim = xy.columns # 날짜, 시가, 고가, 저가, 종가, 거래량
hiddenDim = 10 
outputDim = 1
lr = 0.01
iterations = 500

trainSize = int(len(xy)*0.7)
trainSet = xy_scaled_df[0:trainSize]
testSet = xy_scaled_df[trainSize-seqLength:]

trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

In [69]:
# # 7일간의 5가지 데이터(시가, 종가, 고가, 저가, 거래량)를 받아와서 
# # 바로 다음 날의 종가를 예측하는 모델로 구성

# def buildDataSet(timeSeries, seqLength):
#     xdata = []
#     ydata = [] 
#     for i in range(0,(timeSeries)-seqLength):
#         tx = timeSeries[i:i+seqLength,:-1]
#         ty = timeSeries[i+seqLength,[-1]]
#         xdata.append(tx)
#         ydata.append(ty)
#     return np.array(xdata), np.array(ydata)

In [75]:
from sklearn.model_selection import train_test_split

In [83]:
#train/test size 설정
train_size = int(len(xy_scaled_df)*0.6)
test_size = int(len(xy_scaled_df)*0.3) +train_size

#train/test/validation set 나누기
train_x = np.array(xy_scaled_df[:train_size])
train_y = np.array(xy_scaled_df['종가'][:train_size])

test_x =np.array(xy_scaled_df[train_size:test_size])
test_y = np.array(xy_scaled_df['종가'][train_size:test_size])

validation_x = np.array(xy_scaled_df[test_size:])
validation_y = np.array(xy_scaled_df['종가'][test_size:])

In [84]:
# 모델 구성 

# First, let's define a RNN Cell, as a layer subclass.
from tensorflow.python import keras
from tensorflow.python.keras import layers

In [85]:
model = keras.Sequential()

model.add(layers.SimpleRNN(units=10, 
                           activation='tanh',
                           input_shape=[7,4]))

model.add(layers.Dense(1))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 10)                150       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________


In [86]:
# 모델 학습과정 설정 
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# 모델 트레이닝 
hist = model.fit(train_x, train_y, epochs=1000, batch_size=16)

Epoch 1/1000


ValueError: in user code:


    ValueError: Input 0 of layer sequential_4 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 5)


In [ ]:
# 모델 테스트 
res = model.evaluate(test_x, test_y, batch_size=16)
print("loss",res[0],"mae",res[1])

In [ ]:
#7 모델 사용
xhat = test_x
yhat = model.predict(xhat)

print("Evaluate : {}".format(np.average((yhat - testY)**2)))

In [ ]:
# 원래 값으로 되돌리기 
predict1 = back_MinMax(xy_scaled_df[trainSize-seqLength:,[-1]],yhat)
actual = back_MinMax(xy_scaled_df[trainSize-seqLength:,[-1]],testY)
print("예측값",predict1)
print("실제값",actual)

print(predict1.shape)
print(actual.shape)

In [ ]:
%matplotlib notebook
plt.figure()
plt.plot(predict[:30], label = "predict_RNN")
plt.plot(actual[:30],label = "actual")

plt.legend(prop={'size': 20})

In [ ]:
# LSTM Model
model2 = keras.Sequential()

model2.add(layers.LSTM(units=10, 
                           activation='tanh',
                           input_shape=[7,4]))

model2.add(layers.Dense(1))

model2.summary()

In [ ]:
# 모델 학습과정 설정 
model2.compile(loss='mse', optimizer='adam', metrics=['mae'])

# 모델 트레이닝 
hist = model2.fit(train_x, train_y, epochs=100, batch_size=16)

In [ ]:
# 모델 테스트 
res2 = model2.evaluate(test_x, test_y, batch_size=16)
print("loss",res2[0],"mae",res2[1])

In [ ]:
#7 모델 사용
xhat = test_x
yhat = model2.predict(xhat)
print(testY)
print(yhat)

print("Evaluate : {}".format(np.average((yhat - testY)**2)))

In [ ]:
# 원래 값으로 되돌리기 
predict2 = back_MinMax(xy_scaled_df[trainSize-seqLength:,[-1]],yhat)
actual = back_MinMax(xy_scaled_df[trainSize-seqLength:,[-1]],testY)
print("예측값",predict2)
print("실제값",actual)

In [ ]:
%matplotlib notebook
plt.figure()
plt.plot(predict2[:30], label = "predict_LSTM")
plt.plot(actual[:30],label = "actual")

plt.legend(prop={'size': 20})

In [ ]:
%matplotlib notebook
plt.figure()
day=30
plt.plot(predict1[:day], label = "RNN")
plt.plot(predict2[:day], label = "LSTM")
plt.plot(actual[:day],label = "actual")

plt.legend(prop={'size': 20})
